In [1]:
import d2lzh as d2l
from mxnet import gluon, init
from mxnet.gluon import data as gdata, loss as gloss, model_zoo
import os

# 数据准备好，挂载好位置
# import os
# os.chdir("/content/drive/My Drive/ipynb/kaggle")

# 数据集位置,精简了一下，弄5个类。一共5k张图
data_dir = 'food5'

# 用ImageFolderDataset来读取数据
train_imgs = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, 'train'))
test_imgs = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, 'test'))

# 指定RGB三个通道的均值和方差来将图像通道归一化
normalize = gdata.vision.transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_augs = gdata.vision.transforms.Compose([
    gdata.vision.transforms.RandomResizedCrop(224),
    gdata.vision.transforms.RandomFlipLeftRight(),
    gdata.vision.transforms.ToTensor(),
    normalize])

test_augs = gdata.vision.transforms.Compose([
    gdata.vision.transforms.Resize(256),
    gdata.vision.transforms.CenterCrop(224),
    gdata.vision.transforms.ToTensor(),
    normalize])

# 定义和初始化模型，pretrained=True来自动下载并加载预训练的模型参数
pretrained_net = model_zoo.vision.resnet34_v2(pretrained=True)

finetune_net = model_zoo.vision.resnet34_v2(classes=5) #classes表示类别，其实有点好奇这个参数是传到**里去的（元组）
# 固定features中的参数：
# finetune_net.features.collect_params().setattr('grad_req', 'null')
finetune_net.features = pretrained_net.features
finetune_net.output.initialize(init.Xavier())
# output中的模型参数将在迭代中使用10倍大的学习率,看了源码后。。还有这种高级功能！
finetune_net.output.collect_params().setattr('lr_mult', 10)
print(11)

def train_fine_tuning(net, learning_rate, batch_size=128, num_epochs=10):  # 跑大的数据集就多跑几轮
    train_iter = gdata.DataLoader(
        train_imgs.transform_first(train_augs), batch_size, shuffle=True) # train_augs是前面做的图像通道归一化
    test_iter = gdata.DataLoader(
        test_imgs.transform_first(test_augs), batch_size)
    print(12)
    ctx = d2l.try_all_gpus()
    net.collect_params().reset_ctx(ctx)
    net.hybridize()
    loss = gloss.SoftmaxCrossEntropyLoss()
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {
        'learning_rate': learning_rate, 'wd': 0.001})
    d2l.train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs)

train_fine_tuning(finetune_net, 0.01)

11
12
training on [cpu(0)]
epoch 1, loss 2.9952, train acc 0.504, test acc 0.749, time 928.0 sec
epoch 2, loss 0.6182, train acc 0.787, test acc 0.773, time 871.4 sec
epoch 3, loss 0.4904, train acc 0.825, test acc 0.810, time 850.1 sec


KeyboardInterrupt: 